# Error Analysis

This notebook, takes the predictions and gound truth files to identify the incorrect predictions and makes a combined image of the incorrect prediction (on the left), the ground truth (on the right) and the Full Phrase text (in black with red background)

In [1]:
predictions = []
test_data = []
test_labels = []

Reading the prediction and ground truth files

In [3]:
pred_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/pred_test_FullSentence.txt"
data_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/test_data.txt"
label_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/test_label.txt"
# pred_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/dcxl_zshot/dcxl_zshot_FullSentence.txt"
# data_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/metaclip/test_data.txt"
# label_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/metaclip/test_label.txt"
with open(pred_path) as f:
    predictions = f.readlines()

with open(data_path) as f:
    test_data = f.readlines()

with open(label_path) as f:
    test_labels = f.readlines()


In [4]:
predictions = [x.split("\t")[0] for x in predictions]


In [5]:
predictions[0]

'image.2166.jpg'

In [6]:
# test_data = [x.split("\t")[1] for x in test_data ]
temp = []
for t in test_data:
    tt = t.split("\t")
    try:
        ttt = tt[1]
        temp.append(ttt)
    except:
        print(tt)

In [7]:
test_data = temp

In [8]:
test_data[0]

'football goal'

In [9]:
test_labels = [x.strip("\n") for x in test_labels]

In [10]:
test_labels[0]

'image.2166.jpg'

In [10]:
# these indexes had nan values for Full Phrase, thus they were omitted
# test_labels.pop(40)
# test_labels.pop(367)
# test_data.pop(40)
# test_data.pop(367)

'hair bun'

Function to take in input images and text and make a collage to have a comparative view

In [11]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
listofimages=['Image2.jpg', 'Image6.jpg']

def create_collage(width, height, listofimages, img_name,error_img=True):
    img_name = img_name.replace(" ","-")
    cols = 2
    rows = 1
    thumbnail_width = width//cols
    thumbnail_height = height//rows
    size = thumbnail_width, thumbnail_height
    new_im = Image.new('RGB', (width, height))
    ims = []
    for p in listofimages:
        im = Image.open(p)
        im.thumbnail(size)
        ims.append(im)
    i = 0
    x = 0
    y = 0
    for col in range(cols):
        for row in range(rows):
            # print(i, x, y)
            new_im.paste(ims[i], (x, y))
            i += 1
            y += thumbnail_height
        x += thumbnail_width
        y = 0
    draw = ImageDraw.Draw(new_im)
    # draw.text(, img_name, fill=(255, 0, 0))
    position = (200, 180)
    left, top, right, bottom = draw.textbbox(position, img_name)
    if error_img:
        draw.rectangle((left-5, top-5, right+5, bottom+5), fill="red")
    else:
        draw.rectangle((left-5, top-5, right+5, bottom+5), fill="green")
    draw.text(position, img_name, fill="black",stroke_width=50)
    if error_img:
        new_im.save('/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/errors/'+img_name+".jpg")
    else:
        new_im.save('/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/metaclip/good_'+img_name+".jpg")

# create_collage(450, 200, listofimages)

Reading predictions and making collages for the incorrect predictions

In [97]:
test_img_path = '/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/semeval-2023-task-1-V-WSD-train-v1/val_v1/test_images_resized/'
for i,n in enumerate(predictions):
    if predictions[i] != test_labels[i]:
    #     # print(predictions[i])
    #     # print(test_labels[i])
    #     # print(test_data[i])
        create_collage(450, 200, [test_img_path+predictions[i],test_img_path+test_labels[i]],test_data[i],error_img=True)
    if predictions[i] == test_labels[i]:
    #     # print(predictions[i])
    #     # print(test_labels[i])
    #     # print(test_data[i])
        create_collage(450, 200, [test_img_path+predictions[i],test_img_path+test_labels[i]],test_data[i],error_img=True)

copying the test images consisdered in the last iteration from baseline predictions

In [12]:
from glob import glob
import os
import shutil

base_dir = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/error_examples/"

folders = os.listdir(base_dir)[1:]
dest_dir = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/dcxl_zshot/"
for f in folders:
    print(f)
    imgs = glob(base_dir+f+"/*.jpg")
    for im in imgs:
        os.makedirs(os.path.dirname(dest_dir+"from_prev/"+f+"/"), exist_ok=True)
        im_path = im.split("/")[-1]
        try:
            shutil.copy(dest_dir+"errors/"+im_path, dest_dir+"from_prev/"+f+"/"+im_path)
        except FileNotFoundError as e:
            print("correct: "+str(e).split("/")[-1])

famous
semantic
correct: asm-launch.jpg'
correct: reflecting-glass.jpg'
text


### Create a collage for comparision of predictions from different models

In [22]:
# imgs_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/dcxl_zshot/from_prev"

def create_collage_compare(width, height, listofimages,model_types, img_name,save_path):
    # img_name = img_name.replace(" ","-")
    cols = 4
    rows = 2
    thumbnail_width = width//cols
    thumbnail_height = height//rows
    size = thumbnail_width, thumbnail_height
    new_im = Image.new('RGB', (width, height))
    ims = []
    for p in listofimages:
        im = Image.open(p)
        im.thumbnail(size)
        ims.append(im)
    i = 0
    x = 0
    y = 0
    draw = ImageDraw.Draw(new_im)
    for col in range(cols):
        for row in range(rows):
            # print(i, x, y)
            if i <= len(model_types):
                new_im.paste(ims[i], (x, y))
                if i==0:
                    # print("-")
                    left, top, right, bottom = draw.textbbox((x,y), "orig")
                    draw.rectangle((left-5, top-5, right+5, bottom+5), fill="green")
                    draw.text((x,y), "orig", fill="black",stroke_width=60)
                else:
                    # print(i-1)
                    left, top, right, bottom = draw.textbbox((x,y), model_types[i-1])
                    draw.rectangle((left-5, top-5, right+5, bottom+5), fill="yellow")
                    draw.text((x,y), model_types[i-1], fill="black",stroke_width=60)
                i += 1
                y += thumbnail_height
            
        x += thumbnail_width
        y = 0
    
    # draw.text(, img_name, fill=(255, 0, 0))
    position = (1700, 850)
    left, top, right, bottom = draw.textbbox(position, img_name)
    draw.rectangle((left-5, top-5, right+5, bottom+5), fill="red")
    draw.text(position, img_name, fill="black",stroke_width=50)
    new_im.save(save_path+img_name+".jpg")


get predictions accross models

In [23]:
# folders = ["metaclip","metaclip2_wa","dcxl_zshot","metaclip_zshot"]  
# folders = ["clip_base","clip_base_ft","clip_base_ft_aug", "clip_base_ft_noloss", "meta_clip_small", "metaclip_sm_ft_submissions"]
# folder_paths = []  
all_preds = []
for folder in folders:
    # pred_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/"+folder+"/"
    pred_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/small_submissions/"+folder+"/"
    # folder_paths.append(pred_path+"errors/")
    pred_path= glob(pred_path+"*_FullSentence.txt")[0]
    print(pred_path)
    with open(pred_path) as f:
        predictions = f.readlines()

    predictions = [x.split("\t")[0] for x in predictions]
    all_preds.append(predictions)
    print(predictions)

/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/small_submissions/clip_base/val_submission_FullSentence.txt
['image.2166.jpg', 'image.4429.png', 'image.4432.jpg', 'image.1435.jpg', 'image.4455.jpg', 'image.4464.jpg', 'image.4471.jpg', 'image.4476.jpg', 'image.4495.jpg', 'image.4499.jpg', 'image.4506.jpg', 'image.4521.jpg', 'image.4527.jpg', 'image.4542.jpg', 'image.4551.jpg', 'image.4562.jpg', 'image.4570.jpg', 'image.4573.jpg', 'image.4588.jpg', 'image.4592.jpg', 'image.4604.jpg', 'image.4611.jpg', 'image.4615.jpg', 'image.4631.jpg', 'image.4638.jpg', 'image.4642.jpg', 'image.4654.jpg', 'image.4664.jpg', 'image.4668.jpg', 'image.4678.jpg', 'image.4695.jpg', 'image.2003.jpg', 'image.4705.jpg', 'image.4720.jpg', 'image.4727.jpg', 'image.4735.jpg', 'image.668.jpg', 'image.4758.jpg', 'image.4761.jpg', 'image.1910.jpg', 'image.4794.jpg', 'image.2150.jpg', 'image.4809.jpg', 'image.1295.jpg', 'image.4826.jpg', 'image.2962.jpg', 'image.4840.jpg', 'image.4850.jpg', 'image.4857.jpg', 


#### Create collage for the 3 domains of error analysed in Baseline

In [24]:
test_img_path = '/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/semeval-2023-task-1-V-WSD-train-v1/val_v1/test_images_resized/'

text_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/error_examples/text/"
semantic_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/error_examples/semantic/"
famous_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/baseline/error_examples/famous/"

# Literal text preferance  
# save_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/collated/text/"
save_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/collated_small/text/"
test_imgs = glob(text_path+"*.jpg") 
test_imgs = [j.split("/")[-1].strip(".jpg") for j in test_imgs]
# print(len(test_imgs))
for i in range(len(test_labels)):
    # print(test_data[i])
    # for j in test_imgs:
        # img_name = j.split("/")[-1].strip(".jpg")
        # print(img_name)
        # print(i)
    test_img_name = test_data[i].replace(" ","-")
    if  test_img_name in test_imgs:
        # print("creating")
        print(test_img_name)
        print(i)
        create_collage_compare(1800, 900, [test_img_path+test_labels[i]]+[test_img_path+p[i] for p in all_preds],folders,test_img_name,save_path)

# semantic interpretation challenge
# save_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/collated/semantic/"
save_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/collated_small/semantic/"
test_imgs = glob(semantic_path+"*.jpg") 
test_imgs = [j.split("/")[-1].strip(".jpg") for j in test_imgs]
print(len(test_imgs))
for i in range(len(test_labels)):
    # print(test_data[i])
    # for j in test_imgs:
        # img_name = j.split("/")[-1].strip(".jpg")
        # print(img_name)
        # print(i)
    test_img_name = test_data[i].replace(" ","-")
    if  test_img_name in test_imgs:
        # print("creating")
        print(test_img_name)
        print(i)
        create_collage_compare(1800, 900, [test_img_path+test_labels[i]]+[test_img_path+p[i] for p in all_preds],folders,test_img_name,save_path)

# Prominent word bias
# save_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/collated/famous/"
save_path = "/Users/jeelshah/Documents/Fall 2023/ANLP/project/bla/VWSD/Visual_Word_Sense_Disambiguation/predictions/collated_small/famous/"
test_imgs = glob(famous_path+"*.jpg") 
test_imgs = [j.split("/")[-1].strip(".jpg") for j in test_imgs]
# print(len(test_imgs))
for i in range(len(test_labels)):
    # print(test_data[i])
    # for j in test_imgs:
        # img_name = j.split("/")[-1].strip(".jpg")
        # print(img_name)
        # print(i)
    test_img_name = test_data[i].replace(" ","-")
    if  test_img_name in test_imgs:
        # print("creating")
        print(test_img_name)
        print(i)
        try:
            create_collage_compare(1800, 900, [test_img_path+test_labels[i]]+[test_img_path+p[i] for p in all_preds],folders,test_img_name,save_path)
        except FileNotFoundError:
            print(f"correct: {test_img_name}")

blue-mood
58
wellington-shoe
185
smoking-steam
240
aurora-tale
261
watch-a-movie
276
river-stream
331
scorpio-stinger
396
8
asm-launch
15
reflecting-glass
20
ammunition-shot
139
alpha-star
295
torch-battery
361
battery-shoot
391
living-sponge
404
bay-shore
423
tea-herb
29
amit-game
66
bay-sound
141
teddy-garment
296
alexander-leaves
305
beaver-hat
363
